In [4]:
#!/usr/bin/env python

#environment setting
import pandas as pd
import sqlite3
import re
# Library Import and Environment Setting needed for Logistic Regression

import numpy as np
import matplotlib.pyplot as plt 
from matplotlib.backends.backend_pdf import PdfPages
%matplotlib inline

plt.rc("font", size=16)
import seaborn as sns
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)

import os
import os.path
import glob
import re

import json
import time

from collections import defaultdict
from collections import Counter

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
import nltk.data
nltk.downloader.download('vader_lexicon')
nltk.downloader.download('stopwords')

# from nltk.sentiment.vader import SentimentIntensityAnalyzer

! pip install profanity-check
from profanity_check import predict, predict_prob
# https://github.com/vzhou842/profanity-check

!pip install filelock --ignore-installed

import argparse

!pip install coverage --ignore-installed
!pip install langdetect
from langdetect import detect

!pip install googletrans
from googletrans import Translator
translator = Translator()


# !pip install py-translator
# from py_translator import Translator

# !pip install translate

# from translate import Translator
# translator = Translator(to_lang='en')

import requests, uuid

# import goslate

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/alexpapaioannou/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alexpapaioannou/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


  Using cached https://files.pythonhosted.org/packages/d7/ca/3c74396a9ed8a4cfab5459800edeef9a1269591cb21f5a49bd71a49c5fa2/filelock-3.0.10-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/a8/39/5334b42cc81a40d50901eca26f4fac4480b44ac318db55d2b621d0aaca09/coverage-4.5.3-cp37-cp37m-macosx_10_13_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/59/59/4bc44158a767a6d66de18c4136c8aa90491d56cc951c10b74dd1e13213c9/langdetect-1.0.7.zip
  Stored in directory: /Users/alexpapaioannou/Library/Caches/pip/wheels/ec/0c/a9/1647275e7ef5014e7b83ff30105180e332867d65e7617ddafe
Successfully built langdetect
  Using cached https://files.pythonhosted.org/packages/fd/f0/a22d41d3846d1f46a4f20086141e0428ccc9c6d644aacbfd30990cf46886/googletrans-2.4.0.tar.gz
  Stored in directory: /Users/alexpapaioannou/Library/Caches/pip/wheels/50/d6/e7/a8efd5f2427d5eb258070048718fa56ee5ac57fd6f53505f95
Successfully built googletrans


In [51]:
'222~All This Time Still Falling out of Love [Album Version]~Erasure.txt'.split(".txt")[0]

'222~All This Time Still Falling out of Love [Album Version]~Erasure'

In [ ]:
%time
    
#     parser = argparse.ArgumentParser('Parses the given directory of lyrics')
#     parser.add_argument('dir_given', help='Directory of the lyrics')
#     args = parser.parse_args()
#     dir_given = args.dir_given

#Simulated 'Given' directory, needs to be changed to input from command line kss0416
dir_given = '/Users/alexpapaioannou/Dropbox/Personal Things/Personal/Studies Related/NY/COLUMBIA UNIVERSITY/IEOR/MSMSE/Courses/IEOR 4501 E 001 TOOLS FOR ANALYTICS/TOOLS_2019/Project_2019/Lyrics'

#Need to change path below to start in lyrics directory kss0416
os.chdir(dir_given)

def artists_list()->list:
    artists_list_ = []
    for raw_filename in os.listdir(dir_given):
        if raw_filename.endswith('.txt'):
            try:
                artist_name = raw_filename.split("~")[2].split(".txt")[0]
                if artist_name not in artists_list_:
                    artists_list_.append(artist_name)
            except IndexError:
                pass
    return artists_list_

def raw_filenames_list()->list:
    raw_filenames_list_ = []
    for raw_filename in os.listdir(dir_given):
        if raw_filename.endswith('.txt'):
            raw_filenames_list_.append(raw_filename)
    return raw_filenames_list_

def artist_s_songs_list(str)->list:
    artist_s_songs_list_ = []
    for raw_filename in raw_filenames_list('.txt'):
        if raw_filename.endswith(str+'.txt'):
            artist_s_songs_list_.append(raw_filename)
    return artist_s_songs_list_

def artist_s_cleaned_songs_list(str)->list:
    artist_s_cleaned_songs_list_ = []
    for cleaned_raw_filename in os.listdir(dir_given + '/Cleaned_Songs/'):
        if cleaned_raw_filename.endswith(str + '.txt') is True:
            if cleaned_raw_filename.endswith(str + '.txt') not in artist_s_cleaned_songs_list_:
                artist_s_cleaned_songs_list_.append(cleaned_raw_filename)
    return artist_s_cleaned_songs_list_

artist_s_cleaned_songs_list("The Beatles")

def song_cleaning():
    for artist in artists_list:
        for song in artist_s_songs_list(artist):
            f = open('Lyrics/' + song, 'rb')
            all_words = ''
            for sentence in f.readlines():
                this_sentence = sentence.decode('utf-8')
                all_words += this_sentence
            #remove identifiers like chorus, verse, etc
            all_words = re.sub(r'[\(\[],.*?[\)\]]', '', all_words)
            #remove empty lines
            all_words = os.linesep.join([s for s in all_words.splitlines() if s])
            f.close()
            directory = "Lyrics/Cleaned_Songs"
            f = open(os.path.join(directory, 'cleaned_' + song ), "wb")
            f.write(all_words.encode('utf-8'))
            f.close()

# def song_translating(artists_list):
#     for artist in set(artists_list):
#         for song in artist_s_songs_list(artist):
#             #Changed below to base on single given directory
#             f = open(dir_given + r'/Cleaned_Songs/cleaned_' +  song, 'rb')
#             all_words = ''
#             cleaned_songs_directory = dir_given + "/Cleaned_Songs/"

#             for sentence in f.readlines():
#                 this_sentence = sentence.decode('utf-8')
#                 try:
#                     if translator.detect(this_sentence).lang == 'en':
#                         all_words += this_sentence
#                     else:
#                         translation = translator.translate(this_sentence)
#                         all_words += translation.text
#                 except:
#                     print('exception')
#             f = open(cleaned_songs_directory + 'cleaned_' + song, 'wb')
#             f.write(all_words.encode('utf-8'))
#             f.close()
#     return

# song_translating(artists_list())
            
def id_song_to_be_scored(song_to_be_scored):

    f = open(dir_given + '/Cleaned_Songs/' + song_to_be_scored , 'rb')
#     cleaned_song_name = song_to_be_scored.split("cleaned_")[1].split("~")[1]
    id_song_to_be_scored = song_to_be_scored.split("cleaned_")[1].split("~")[0]

    return id_song_to_be_scored

a = id_song_to_be_scored('cleaned_688~I Wanna Be Loved~Buy This Song.txt')

print('id:', a)

def artist_song_to_be_scored(song_to_be_scored):

    f = open(dir_given + '/Cleaned_Songs/' + song_to_be_scored , 'rb')
    artist_song_to_be_scored = song_to_be_scored.split("~")[2].split(".txt")[0]
    
    return artist_song_to_be_scored

b = artist_song_to_be_scored('cleaned_688~I Wanna Be Loved~Buy This Song.txt')

print('artist:', b)

def title_song_to_be_scored(song_to_be_scored):

    f = open(dir_given + '/Cleaned_Songs/' + song_to_be_scored , 'rb')
    title_song_to_be_scored = song_to_be_scored.split("~")[1].split(".txt")[0]
    
    return title_song_to_be_scored

c = title_song_to_be_scored('cleaned_688~I Wanna Be Loved~Buy This Song.txt')

print('title:', c)

def profanity_score_min_max():
    
    profanity_score_list_ = []

    for artist in set(artists_list()):   
        
        for song in artist_s_cleaned_songs_list(artist):
            
            raw_text = "" 
            f = open(dir_given + '/Cleaned_Songs/' + song , 'rb')
            for sentence in f.readlines():
                this_sentence = sentence.decode('utf-8')
                raw_text += this_sentence

            song_lyrics_for_profanity_check_ = [raw_text]
            profanity_check = predict_prob(song_lyrics_for_profanity_check_)
            profanity_score = float(' '.join(map(str, profanity_check)))
            profanity_score_list_.append(profanity_score)
                
    min_profanity_score_list_ = round(min(profanity_score_list_),2)
    max_profanity_score_list_ = round(max(profanity_score_list_),2)

    return min_profanity_score_list_, max_profanity_score_list_

d,e = profanity_score_min_max()

print('profanity score: min=', d, ',', 'max=', e)

profanity_score_min, profanity_score_max = profanity_score_min_max()

def profanity_score(song_to_be_scored):

    raw_text = "" 
    f = open(dir_given + '/Cleaned_Songs/' + song_to_be_scored , 'rb')
    for sentence in f.readlines():
        this_sentence = sentence.decode('utf-8')            
        raw_text += this_sentence
    
    song_lyrics_for_profanity_check_of_song_to_be_scored = [raw_text]
    profanity_check = predict_prob(song_lyrics_for_profanity_check_of_song_to_be_scored)
    profanity_score_of_song_to_be_scored = float(' '.join(map(str, profanity_check)))

    regularization_step = (profanity_score_of_song_to_be_scored - profanity_score_min)/(profanity_score_max - profanity_score_min) 
    profanity_score_of_song_to_be_scored_regularized = 1*regularization_step + 0*(1-regularization_step)

    return round(profanity_score_of_song_to_be_scored_regularized,4)

f = profanity_score('cleaned_688~I Wanna Be Loved~Buy This Song.txt')

print('profanity score=', f)

def love_score_min_max():
    love_words_list_ = [
                   'adore', 'adores', 'adorable', 'affection', 'amour', 'angel', 'bliss', 
#                    'care', 'caring', 'chocolate', 'companion', 'compassion', 'concern', 
#                    'darling', 'dear', 'desire', 'devotion', 'endearment', 'family', 
#                    'fondness', 'forever', 'friendship', 'fun', 'God', 'happiness', 'happy', 
                   'happily', 'heart', 'hugs', 'husband', 'infatuation', 'inspiration', 
                   'intimacy', 'joy', 'kiss', 'kissed', 'kisses', 
                   'love', 'loves', 'loved', 'loving', 
#                    'loyalty', 'marriage', 'passion', 'relationship', 'romance', 'sex', 
#                    'sweet', 'sweetheart', 'tenderness', 'trust', 'warmth', 'wife'
                    ]
    love_score_list_ = []

    for artist in set(artists_list()):   

        for song in artist_s_cleaned_songs_list(artist):
            raw_text = "" 
            counter_for_love_words = 0
            f = open(dir_given + '/Cleaned_Songs/' + song , 'rb')
            for sentence in f.readlines():
                this_sentence = sentence.decode('utf-8')            
                raw_text += this_sentence
            words = raw_text.split(" ")
            filtered_words = [word for word in words if word not in stopwords.words('english') and len(word) > 1 and word not in ['na','la']] # remove the stopwords
            for item in filtered_words:
                if item in love_words_list_:
                    counter_for_love_words += 1
            len_counter_for_love_words = len(love_words_list_)
            love_score = counter_for_love_words/len_counter_for_love_words
            love_score_rounded = round(love_score,4)
            love_score_list_.append(love_score_rounded) 
            
    min_love_score_list_ = round(min(love_score_list_),4)
    max_love_score_list_ = round(max(love_score_list_),4)

    return min_love_score_list_, max_love_score_list_

g,h = love_score_min_max()

print('love score: min=', g, ',', 'max=', h)

love_score_min, love_score_max = love_score_min_max()

def love_score(song_to_be_scored):
    
    love_words_list_ = [
                   'adore', 'adores', 'adorable', 'affection', 'amour', 'angel', 'bliss', 
#                    'care', 'caring', 'chocolate', 'companion', 'compassion', 'concern', 
#                    'darling', 'dear', 'desire', 'devotion', 'endearment', 'family', 
#                    'fondness', 'forever', 'friendship', 'fun', 'God', 'happiness', 'happy', 
                   'happily', 'heart', 'hugs', 'husband', 'infatuation', 'inspiration', 
                   'intimacy', 'joy', 'kiss', 'kissed', 'kisses', 
                   'love', 'loves', 'loved', 'loving', 
#                    'loyalty', 'marriage', 'passion', 'relationship', 'romance', 'sex', 
#                    'sweet', 'sweetheart', 'tenderness', 'trust', 'warmth', 'wife'
                    ]
        
    raw_text = "" 
    f = open(dir_given + '/Cleaned_Songs/' + song_to_be_scored , 'rb')
    counter_for_love_words = 0
    for sentence in f.readlines():
        this_sentence = sentence.decode('utf-8')            
        raw_text += this_sentence
    words = raw_text.split(" ")
    filtered_words = [word for word in words if word not in stopwords.words('english') and len(word) > 1 and word not in ['na','la']] # remove the stopwords
    for item in words:
        if item in love_words_list_:
            counter_for_love_words += 1
    len_counter_for_love_words = len(love_words_list_)
    love_score_of_song_to_be_scored = counter_for_love_words/len_counter_for_love_words
    love_score_of_song_to_be_scored_rounded = round(love_score_of_song_to_be_scored,4)
    
    regularization_step = (love_score_of_song_to_be_scored_rounded - love_score_min)/(love_score_max - love_score_min) 
    love_score_of_song_to_be_scored_rounded_regularized = 1*regularization_step + 0*(1-regularization_step)

    return round(love_score_of_song_to_be_scored_rounded_regularized,4)

i = love_score('cleaned_688~I Wanna Be Loved~Buy This Song.txt')

print('love score=', i)

def mood_score_min_max()->float:

    sid = SentimentIntensityAnalyzer()
    mood_score_list_ = []

    for artist in set(artists_list()):   

        for song in artist_s_cleaned_songs_list(artist):
            raw_text = "" 
            f = open(dir_given + '/Cleaned_Songs/' + song , 'rb')
            for sentence in f.readlines():
                this_sentence = sentence.decode('utf-8')            
                raw_text += this_sentence
            mood_score_uncompounded = sid.polarity_scores(raw_text)
            mood_score_compounded = mood_score_uncompounded['compound']
            mood_score_compounded_rounded = round(mood_score_compounded,2)
            mood_score_list_.append(mood_score_compounded_rounded) 
            
    min_mood_score_list_ = round(min(mood_score_list_),4)
    max_mood_score_list_ = round(max(mood_score_list_),4)

    return min_mood_score_list_, max_mood_score_list_

j,k = mood_score_min_max()

print('mood score: min=', j, ',', 'max=', k)

mood_score_min, mood_score_max = mood_score_min_max()


def mood_score(song_to_be_scored)->float:
    
    sid = SentimentIntensityAnalyzer()
    raw_text = ""

    f = open(dir_given + '/Cleaned_Songs/' + song_to_be_scored , 'rb')
    for sentence in f.readlines():
        this_sentence = sentence.decode('utf-8')
        raw_text += this_sentence
    mood_score_uncompounded_of_song_to_be_scored = sid.polarity_scores(raw_text)
    mood_score_compounded_of_song_to_be_scored = mood_score_uncompounded_of_song_to_be_scored['compound']
    mood_score_compounded_of_song_to_be_scored_rounded = round(mood_score_compounded_of_song_to_be_scored,2)
    
    regularization_step = (mood_score_compounded_of_song_to_be_scored_rounded - mood_score_min)/(mood_score_max - mood_score_min) 
    mood_score_compounded_of_song_to_be_scored_rounded_regularized = 1*regularization_step + 0*(1-regularization_step)
    
    return mood_score_compounded_of_song_to_be_scored_rounded_regularized

l = mood_score('cleaned_688~I Wanna Be Loved~Buy This Song.txt')

print('mood score=', l)

def length_score_min_max():

    length_score_list_ = []

    for artist in set(artists_list()):   

        for song in artist_s_cleaned_songs_list(artist):
                
            num_words = 0
    
            f = open(dir_given + '/Cleaned_Songs/' + song , 'rb')
            for line in f.readlines():
                this_line_wordlist = line.split()
                num_words += len(this_line_wordlist)
            length_score_list_.append(num_words)
            
    min_length_score_list_ = min(length_score_list_)
    max_length_score_list_ = max(length_score_list_)
                
    return min_length_score_list_, max_length_score_list_

m,n = length_score_min_max()

print('length score: min=', m, ',', 'max=', n)

length_score_min, length_score_max = length_score_min_max()


def length_score(song_to_be_scored):
    
    num_words_of_song_to_be_scored = 0
    f = open(dir_given + '/Cleaned_Songs/' + song_to_be_scored , 'rb')
    for line in f.readlines():
        this_line_wordlist = line.split()            
        num_words_of_song_to_be_scored += len(this_line_wordlist)

    length_score_of_song_to_be_scored = num_words_of_song_to_be_scored
    
    regularization_step = (length_score_of_song_to_be_scored - length_score_min)/(length_score_max - length_score_min) 
    length_score_of_song_to_be_scored_regularized = 1*regularization_step + 0*(1-regularization_step)

    return round(length_score_of_song_to_be_scored_regularized,2)

o = length_score('cleaned_688~I Wanna Be Loved~Buy This Song.txt')

print('length score=', o)

def complexity_score_min_max():

    complexity_score_list_ = []

    for artist in set(artists_list()):   

        for song in artist_s_cleaned_songs_list(artist):
                    
            num_words = 0
            words_of_lyrics = []

            f = open(dir_given + '/Cleaned_Songs/' + song , 'rb')
            for line in f.readlines():
                this_line_wordlist = line.split()            
                num_words += len(this_line_wordlist)
                for word_ in this_line_wordlist:
                    words_of_lyrics.append(word_)
            
            filtered_words = [word for word in words_of_lyrics if word not in stopwords.words('english') and len(word) > 1 and word not in ['na','la']] # remove the stopwords
            unique_number_of_words = len(set(filtered_words))
            number_of_words = len(words_of_lyrics)
            complexity_score = unique_number_of_words/number_of_words
            complexity_score = round(complexity_score,2)
            complexity_score_list_.append(complexity_score)
                        
    min_complexity_score_list_ = min(set(complexity_score_list_))
    max_complexity_score_list_ = max(set(complexity_score_list_))
                                    
    return min_complexity_score_list_, max_complexity_score_list_

p,q = complexity_score_min_max()

print('complexity score: min=', p, ',', 'max=', q)

complexity_score_min, complexity_score_max = complexity_score_min_max()


def complexity_score(song_to_be_scored):
    
    num_words_of_song_to_be_scored = 0
    words_of_lyrics_of_song_to_be_scored = []
    
    f = open(dir_given + '/Cleaned_Songs/' + song_to_be_scored , 'rb')
    for line in f.readlines():
        this_line_wordlist = line.split()            
        num_words_of_song_to_be_scored += len(this_line_wordlist)
        for word_ in this_line_wordlist:
            words_of_lyrics_of_song_to_be_scored.append(word_)
    
    filtered_words_of_song_to_be_scored = [word for word in words_of_lyrics_of_song_to_be_scored if word not in stopwords.words('english') and len(word) > 1 and word not in ['na','la']] # remove the stopwords
    unique_number_of_words_of_song_to_be_scored = len(set(filtered_words_of_song_to_be_scored))
    number_of_words_of_song_to_be_scored = len(words_of_lyrics_of_song_to_be_scored)
    complexity_score_of_song_to_be_scored = unique_number_of_words_of_song_to_be_scored/number_of_words_of_song_to_be_scored
    complexity_score_of_song_to_be_scored = round(complexity_score_of_song_to_be_scored,2)
    
    regularization_step = (complexity_score_of_song_to_be_scored - complexity_score_min)/(complexity_score_max - complexity_score_min) 
    complexity_score_of_song_to_be_scored_regularized = 1*regularization_step + 0*(1-regularization_step)

    return round(complexity_score_of_song_to_be_scored_regularized, 2)

r = complexity_score('cleaned_688~I Wanna Be Loved~Buy This Song.txt')

print('complexity score=', r)

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.25 µs
id: 688
artist: Buy This Song
title: I Wanna Be Loved
profanity score: min= 0.0 , max= 1.0
profanity score= 0.0317


In [ ]:
%time

dict_for_json = {}

for artist in set(artists_list()):   
    for song in artist_s_cleaned_songs_list(artist):
        dict_for_json_song = {}
        dict_for_json_song["id"] = id_song_to_be_scored(song)
        dict_for_json_song["artist"] = artist
        dict_for_json_song["title"] = title_song_to_be_scored(song)        
        dict_for_json_song["kids_safe"] = profanity_score(song)
        dict_for_json_song["love"] = love_score(song)
        dict_for_json_song["mood"] = mood_score(song)
        dict_for_json_song["length"] = length_score(song)
        dict_for_json_song["complexity"] = complexity_score(song)

print(json.dumps(dict_for_json, indent=4))
len(dict_for_json['characterizations:'])

In [7]:
#                     if translator.detect(this_sentence).lang == 'en':
#                         all_words += this_sentence
#                     else:
#                         translation = translator.translate(this_sentence)
#                         all_words += translation.text
#                 except:
#                     pass
 
#             f.close()
#             #Deleted Directory variable and modified below line to take given directory
#             f = open(dir_given + r'/Cleaned_Songs/cleaned_' +  song, 'wb')
#             f.write(all_words.encode('utf-8'))
#             f.close()
# %time

In [8]:
# %time
# plt.rcParams['figure.figsize'] = (126,190)

# df = pd.DataFrame(columns=('artist', 'words'))
# i=0

# def word_num(artists):
#     print('running')
#     i = 0
#     for artist in set(artists):
#         num_words = 0
#         all_words = ''
#         df.loc[i] = (artist, num_words)
#         i += 1
#         for song in artist_s_cleaned_songs_list(artist):
#             f = open(dir_given + '/Cleaned_Songs/' + song , 'rb')
#             for sentence in f.readlines():
#                 this_sentence = sentence.decode('utf-8')
#                 num_words_this = len(this_sentence.split(" "))
#                 num_words += num_words_this

# word_num(artists)        

In [9]:
# def sentiment(artists):
#     df = pd.DataFrame(columns=('artist', 'pos', 'neu', 'neg'))
#     #Added download for vader lexicon in the import section for below step kss0426
#     sid = SentimentIntensityAnalyzer()
#     # Commas do not affect the SentimentIntensityAnalyzer() (http://www.nltk.org/howto/sentiment.html)
#     i=0

#     num_neutral = 0
#     num_negative = 0
#     num_positive = 0

#     for artist in set(artists):

#         num_neutral = 0
#         num_negative = 0
#         num_positive = 0

#         for song in artist_s_cleaned_songs_list(artist):
#             f = open(dir_given + '/Cleaned_Songs/' + song , 'rb')
#             for sentence in f.readlines():
#                 this_sentence = sentence.decode('utf-8')
#                 comp = sid.polarity_scores(this_sentence)
#                 comp = comp['compound']
#                 if comp >= 0.5:
#                     num_positive += 1
#                 elif comp > -0.5 and comp < 0.5:
#                     num_neutral += 1
#                 else:
#                     num_negative += 1

#         num_total = num_negative + num_neutral + num_positive
#         if num_total != 0:
#             percent_negative = (num_negative/float(num_total))*100
#             percent_neutral = (num_neutral/float(num_total))*100
#             percent_positive = (num_positive/float(num_total))*100
#             df.loc[i] = (artist, percent_positive, percent_neutral, percent_negative)
#             i+=1
#         else:
#             percent_negative = 0
#             percent_neutral = 0
#             percent_positive = 0
#             df.loc[i] = (artist, percent_positive, percent_neutral, percent_negative)
#             i+=1
    
# sentiment(artists)

In [79]:
# plt.rcParams['figure.figsize'] = (126,190)

# df = pd.DataFrame(columns=('artist', 'words'))
# i=0

# def artist_s_cleaned_songs_list(str)->list:
#     artist_s_cleaned_songs_list_ = []
#     for cleaned_raw_filename in os.listdir(dir_given + '/Cleaned_Songs'):
#         if cleaned_raw_filename.endswith(str + '.txt') is True:
#             if cleaned_raw_filename.endswith(str + '.txt') not in artist_s_cleaned_songs_list_:
#                 artist_s_cleaned_songs_list_.append(cleaned_raw_filename)
#     return artist_s_cleaned_songs_list_

# def min_max_numfunc(artists_list):
#     num_words_list_ = []

#     for artist in artists_list:
#         num_words = 0
#         for song in artist_s_cleaned_songs_list(artist):
#             f = open(dir_given + '/Cleaned_Songs/' + song , 'rb')
#             for sentence in f.readlines():
#                 this_sentence = sentence.decode('utf-8')
#                 num_words_this = len(this_sentence.split(" "))
#                 num_words += num_words_this
#         num_words_list_.append(num_words)
        
#     min_num_words_list_ = min(num_words_list_)
#     max_num_words_list_ = max(num_words_list_)
#     return min_num_words_list_, max_num_words_list_
    
# min_num_words_list_, max_num_words_list_ = min_max_numfunc(artists_list())

In [11]:
# def regularize_num(artists):
#     for artist in set(artists):
#         num_words = 0
#         for song in artist_s_cleaned_songs_list(artist):
#             f = open(dir_given + '/Cleaned_Songs/' + song , 'rb')
#             for sentence in f.readlines():
#                 this_sentence = sentence.decode('utf-8')
#                 num_words_this = len(this_sentence.split(" "))
#                 num_words += num_words_this
#             regularization_step = (num_words - min_num_words_list_)/(max_num_words_list_ - min_num_words_list_) 
#             num_words_score = 1*regularization_step + 0*(1-regularization_step)
#             num_words_score = round(num_words_score,2)
    
#     return num_words_score

# num_words_score = regularize_num(artists)

In [12]:
# def lexical_richness(artists):
#     plt.rcParams['figure.figsize'] = (126,190)

#     df = pd.DataFrame(columns=('artist', 'lexicalrichness'))
#     for artist in set(artists):
#         num_words = 0
#         all_words = ''
#         raw_text = ""
#         for song in artist_s_cleaned_songs_list(artist):
#             f = open(dir_given + '/Cleaned_Songs/' + song , 'rb')
#             for sentence in f.readlines():
#                 this_sentence = sentence.decode('utf-8')
#                 raw_text += this_sentence

#         words = raw_text.split(" ")
#         filtered_words = [word for word in words if word not in stopwords.words('english') and len(word) > 1 and word not in ['na','la']] # remove the stopwords

#         a = len(set(filtered_words))
#         b = len(words)

# lexical_richness(artists)

# def unique_words_ratio(song):
#     num_words = 0
#     all_words = ''
#     raw_text = ""    
#     f = open(dir_given + '/Cleaned_Songs/' + song , 'rb')
#     for sentence in f.readlines():
#         this_sentence = sentence.decode('utf-8')
#         raw_text += this_sentence
#     words = raw_text.split(" ")
#     filtered_words = [word for word in words if word not in stopwords.words('english') and len(word) > 1 and word not in ['na','la']] # remove the stopwords

#     unique_number_of_words = len(set(filtered_words))
#     number_of_words = len(words)
#     unique_words_ratio = unique_number_of_words/float(number_of_words)
#     unique_words_ratio = round(unique_words_ratio,2)

#     return unique_words_ratio

# unique_words_ratio('cleaned_688~I Wanna Be Loved~Buy This Song.txt')

In [130]:
# s = complexity_score('cleaned_1001~test~test.txt')

# print('complexity score=', s)

complexity score= 0.58


In [117]:
df = pd.DataFrame(columns=('artist', 'pos', 'neu', 'neg'))
sid = SentimentIntensityAnalyzer()
i=0

# os.chdir('/Users/alexpapaioannou/Dropbox/Personal Things/Studies Related/NY/COLUMBIA UNIVERSITY/IEOR/MSMSE/Courses/IEOR 4501 E 001 TOOLS FOR ANALYTICS/TOOLS_2019/Project_2019/')
os.chdir(dir_given)
num_neutral = 0
num_negative = 0
num_positive = 0
unique_words_list_ = []
love_words_list_ = [
                   'adore', 'adores', 'adorable', 'affection', 'amour', 'angel', 'bliss', 
#                    'care', 'caring', 'chocolate', 'companion', 'compassion', 'concern', 
#                    'darling', 'dear', 'desire', 'devotion', 'endearment', 'family', 
#                    'fondness', 'forever', 'friendship', 'fun', 'God', 'happiness', 'happy', 
                   'happily', 'heart', 'hugs', 'husband', 'infatuation', 'inspiration', 
                   'intimacy', 'joy', 'kiss', 'kisses', 
                   'love', 'loves', 'loved', 'loving', 
#                    'loyalty', 'marriage', 'passion', 'relationship', 'romance', 'sex', 
#                    'sweet', 'sweetheart', 'tenderness', 'trust', 'warmth', 'wife'
                    ]

love_ratio_list_ = []
dict_for_json = {}
with open('data_output.txt', 'w') as fp:
# with open('./data_output.txt', 'w') as f:
    print('{\n      "characterizations": [')
    print('\n             {')
    for artist in set(artists_list()):   

        num_neutral = 0
        num_negative = 0
        num_positive = 0
        
        num_words = 0
        all_words = ''
        raw_text = "" 
        
        counter_for_love_words = 0


#         if len(artist_s_cleaned_songs_list(artist)) == 1:
        
        for song in artist_s_cleaned_songs_list(artist):
            dict_for_json_song = {}
            cleaned_song_name = song.split("cleaned_")[1].split("~")[1]
            id_ = song.split("cleaned_")[1].split("~")[0]
            
            dict_for_json_song["id"] = id_
            dict_for_json_song["artist"] = artist
            dict_for_json_song["title"] = cleaned_song_name
            
            print(f'\n                     "id" = {id_},\n                     "artist" = {artist},') 

            f = open(dir_given + '/Cleaned_Songs/' + song , 'rb')
            for sentence in f.readlines():
                this_sentence = sentence.decode('utf-8')
                
                raw_text += this_sentence
                
                num_words_this = len(this_sentence.split(" "))
                num_words += num_words_this
                
                comp = sid.polarity_scores(this_sentence)
                comp = comp['compound']
                if comp >= 0.5:
                    num_positive += 1
                elif comp > 0 and comp < 0.5:
                    num_neutral += 1
                else:
                    num_negative += 1
                
            sentence_for_profanity_check_ = [this_sentence]
            profanity_check = predict_prob(sentence_for_profanity_check_)
            profanity_score = float(' '.join(map(str, profanity_check)))
            dict_for_json_song["kids_safe"] = round(profanity_score,2)
                
            regularization_step = (num_words - min_num_words_list_)/(max_num_words_list_ - min_num_words_list_) 
            num_words_score = 1*regularization_step + 0*(1-regularization_step)
            num_words_score = round(num_words_score,2)

            num_total = num_negative + num_neutral + num_positive
            if num_total != 0:
                percent_negative = round((num_negative/float(num_total))*100,2)
                percent_neutral = round((num_neutral/float(num_total))*100,2)
                percent_positive = round((num_positive/float(num_total))*100,2)
                df.loc[i] = (artist, percent_positive, percent_neutral, percent_negative)
                i+=1
            else:
                percent_negative = 0
                percent_neutral = 0
                percent_positive = 0
            
            words = raw_text.split(" ")
            filtered_words = [word for word in words if word not in stopwords.words('english') and len(word) > 1 and word not in ['na','la']] # remove the stopwords
            
            a = len(set(filtered_words))
            b = len(words)
            unique_words_ratio = a/float(b)
            unique_words_ratio = round(unique_words_ratio,2)
            unique_words_list_.append(unique_words_ratio)
            
            
            for item in words:
                if item in love_words_list_:
                    counter_for_love_words += 1
            len_counter_for_love_words = len(love_words_list_)
            love_ratio = counter_for_love_words/float(len_counter_for_love_words)
            love_ratio = round(love_ratio,4)
            love_ratio_list_.append(love_ratio) 
            
            min_love_ratio_list_ = min(love_ratio_list_)
            max_love_ratio_list_ = max(love_ratio_list_)
            
            regularization_step = (love_ratio - min_love_ratio_list_)/(max_love_ratio_list_ - min_love_ratio_list_ - 16) 
            love_ratio_score = 1*regularization_step
            love_ratio_score = round(love_ratio_score,2)
            
            dict_for_json_song["love"] = round(love_ratio_score,2)
            dict_for_json_song["mood"] = round(comp,2)
            dict_for_json_song["length"] = num_words_score
            dict_for_json_song["complexity"] = unique_words_ratio
            

#             dict_for_json.setdefault('characterizations:', []).append(dict_for_json_song)

{
      "characterizations": [

             {

                     "id" = 959,
                     "artist" = Conjunto Atardecer,

                     "id" = 645,
                     "artist" = Los Ángeles Azules,

                     "id" = 341,
                     "artist" = The Velvets,

                     "id" = 171,
                     "artist" = Kirk Whalum,

                     "id" = 226,
                     "artist" = Paulina Rubio,

                     "id" = 855,
                     "artist" = James Taylor,

                     "id" = 817,
                     "artist" = Daryl Hall & John Oates,

                     "id" = 812,
                     "artist" = Afta-1,

                     "id" = 961,
                     "artist" = Jerry Vale,

                     "id" = 297,
                     "artist" = Korn,

                     "id" = 601,
                     "artist" = Steely Dan,

                     "id" = 221,
                     "artist" = Zao

In [85]:
# print(dict_for_json)
print(json.dumps(dict_for_json, indent=4))

{}


In [86]:
# with open('data_output.json', 'r') as f:  
#     dict_for_json = json.load(f)
#     print(dict_for_json)

len(dict_for_json['characterizations:'])

KeyError: 'characterizations:'

In [87]:
def love_ratio_score(song)->float:
    
    words = []
    raw_text = "" 
    counter_for_love_words = 0
    
    love_words_list_ = [
                   'adore', 'adores', 'adorable', 'affection', 'amour', 'angel', 'bliss', 
#                    'care', 'caring', 'chocolate', 'companion', 'compassion', 'concern', 
#                    'darling', 'dear', 'desire', 'devotion', 'endearment', 'family', 
#                    'fondness', 'forever', 'friendship', 'fun', 'God', 'happiness', 'happy', 
                   'happily', 'heart', 'hugs', 'husband', 'infatuation', 'inspiration', 
                   'intimacy', 'joy', 'kiss', 'kisses', 
                   'love', 'loves', 'loved', 'loving', 
#                    'loyalty', 'marriage', 'passion', 'relationship', 'romance', 'sex', 
#                    'sweet', 'sweetheart', 'tenderness', 'trust', 'warmth', 'wife'
                    ]
    
    love_ratio_list_ = []

    f = open('Lyrics/Cleaned_Songs/' + song , 'rb')
    for sentence in f.readlines():
        this_sentence = sentence.decode('utf-8')
        raw_text += this_sentence

    words = raw_text.split(" ")
    filtered_words = [word for word in words if word not in stopwords.words('english') and len(word) > 1 and word not in ['na','la']] # remove the stopwords

    for item in words:
        if item in love_words_list_:
            counter_for_love_words += 1
    len_counter_for_love_words = len(love_words_list_)
    love_ratio = counter_for_love_words/float(len_counter_for_love_words)
    love_ratio = round(love_ratio,4)
    love_ratio_list_.append(love_ratio)
    
    for artist in set(artists):   
        
        for song in artist_s_cleaned_songs_list(artist):
            cleaned_song_name = song.split("cleaned_")[1].split("~")[1]
            id_ = song.split("cleaned_")[1].split("~")[0]
            f = open('Lyrics/Cleaned_Songs/' + song , 'rb')
            for sentence in f.readlines():
                this_sentence = sentence.decode('utf-8')
                
                raw_text += this_sentence
            
            words = raw_text.split(" ")
            filtered_words = [word for word in words if word not in stopwords.words('english') and len(word) > 1 and word not in ['na','la']] # remove the stopwords
            
            for item in words:
                if item in love_words_list_:
                    counter_for_love_words += 1
            len_counter_for_love_words = len(love_words_list_)
            love_ratio = counter_for_love_words/float(len_counter_for_love_words)
            love_ratio = round(love_ratio,4)
            love_ratio_list_.append(love_ratio) 
            
    min_love_ratio_list_ = min(love_ratio_list_)
    max_love_ratio_list_ = max(love_ratio_list_)

    regularization_step = (love_ratio - min_love_ratio_list_)/(max_love_ratio_list_ - min_love_ratio_list_) 
    love_ratio_score = 1*regularization_step + 0*(1-regularization_step)
    love_ratio_score = round(love_ratio_score,1)
    return love_ratio_score

In [88]:
# a = love_ratio_score('cleaned_688~I Wanna Be Loved~Buy This Song.txt')
# print(a)

In [89]:
# %time

# def song_translating(artists):
#     for artist in set(artists):
#         for song in artist_s_songs_list(artist):
#     #Changed below to base on single given directory kss0416
#             f = open(dir_given + r'/Cleaned_Songs/cleaned_' +  song, 'rb')
#             all_words = ''
#             for sentence in f.readlines():
                
#                 this_sentence = sentence.decode('utf-8')
#                 try:
#                     if translator.detect(this_sentence).lang == 'en':
#                         all_words += this_sentence
#                     else:
#                         translation = translator.translate(this_sentence)
#                         all_words += translation.text

#     #                         from time import sleep
#     #                         translator = Translator()
#     #                         translation = translator.translate(this_sentence).text
#     #                         all_words += translation
#     #                         sleep(1)
#     #                     gs = goslate.Goslate()
#     #                     translatedText = gs.translate(this_sentence,'en')
#     #                     print(translatedText)
#     #                     all_words += this_sentence
#                 except:
#                     pass
 
#             f.close()
#             #Deleted Directory variable and modified below line to take given directory
#             f = open(os.path.join(dir_given + '/Cleaned_Songs', 'cleaned_' + song ), "wb")
#             f.write(all_words.encode('utf-8'))
#             f.close()

# song_translating(artists)

In [90]:
def love_score_min_max():
    love_words_list_ = [
                   'adore', 'adores', 'adorable', 'affection', 'amour', 'angel', 'bliss', 
#                    'care', 'caring', 'chocolate', 'companion', 'compassion', 'concern', 
#                    'darling', 'dear', 'desire', 'devotion', 'endearment', 'family', 
#                    'fondness', 'forever', 'friendship', 'fun', 'God', 'happiness', 'happy', 
                   'happily', 'heart', 'hugs', 'husband', 'infatuation', 'inspiration', 
                   'intimacy', 'joy', 'kiss', 'kisses', 
                   'love', 'loves', 'loved', 'loving', 
#                    'loyalty', 'marriage', 'passion', 'relationship', 'romance', 'sex', 
#                    'sweet', 'sweetheart', 'tenderness', 'trust', 'warmth', 'wife'
                    ]
    love_score_list_ = []

    for artist in set(artists_list()):   

        raw_text = "" 
        
        counter_for_love_words = 0

        for song in artist_s_cleaned_songs_list(artist):
            cleaned_song_name = song.split("cleaned_")[1].split("~")[1]
            id_ = song.split("cleaned_")[1].split("~")[0]
            f = open(dir_given + '/Cleaned_Songs/' + song , 'rb')
            for sentence in f.readlines():
                this_sentence = sentence.decode('utf-8')            
                raw_text += this_sentence
            words = raw_text.split(" ")
            filtered_words = [word for word in words if word not in stopwords.words('english') and len(word) > 1 and word not in ['na','la']] # remove the stopwords
            for item in words:
                if item in love_words_list_:
                    counter_for_love_words += 1
            len_counter_for_love_words = len(love_words_list_)
            love_score = counter_for_love_words/int(len_counter_for_love_words)
            love_score_rounded = round(love_score,4)
            love_score_list_.append(love_score) 
            
    min_love_score_list_ = round(min(love_score_list_),4)
    max_love_score_list_ = round(max(love_score_list_),4)

    return min_love_score_list_, max_love_score_list_

# a=love_score_min_max()

# print(a)

def love_score(song_to_be_scored):
    
    love_words_list_ = [
                   'adore', 'adores', 'adorable', 'affection', 'amour', 'angel', 'bliss', 
#                    'care', 'caring', 'chocolate', 'companion', 'compassion', 'concern', 
#                    'darling', 'dear', 'desire', 'devotion', 'endearment', 'family', 
#                    'fondness', 'forever', 'friendship', 'fun', 'God', 'happiness', 'happy', 
                   'happily', 'heart', 'hugs', 'husband', 'infatuation', 'inspiration', 
                   'intimacy', 'joy', 'kiss', 'kisses', 
                   'love', 'loves', 'loved', 'loving', 
#                    'loyalty', 'marriage', 'passion', 'relationship', 'romance', 'sex', 
#                    'sweet', 'sweetheart', 'tenderness', 'trust', 'warmth', 'wife'
                    ]
        
    raw_text = "" 
    f = open(dir_given + '/Cleaned_Songs/' + song_to_be_scored , 'rb')
    counter_for_love_words = 0
    for sentence in f.readlines():
        this_sentence = sentence.decode('utf-8')            
        raw_text += this_sentence
    words = raw_text.split(" ")
    filtered_words = [word for word in words if word not in stopwords.words('english') and len(word) > 1 and word not in ['na','la']] # remove the stopwords
    for item in words:
        if item in love_words_list_:
            counter_for_love_words += 1
    len_counter_for_love_words = len(love_words_list_)
    love_score_of_song_to_be_scored = counter_for_love_words/int(len_counter_for_love_words)
    love_score_of_song_to_be_scored_rounded = round(love_score_of_song_to_be_scored,4)
    
    love_score_min, love_score_max = love_score_min_max()
    regularization_step = (love_score_of_song_to_be_scored_rounded - love_score_min)/(love_score_max - love_score_min) 
    love_score_of_song_to_be_scored_rounded_regularized = 1*regularization_step + 0*(1-regularization_step)

    return round(love_score_of_song_to_be_scored_rounded_regularized,4)

# b=love_score('cleaned_688~I Wanna Be Loved~Buy This Song.txt')

# print(b)



# print(love_ratio_list_)      # uncomment to see the list with the number of words per song
# print(min(love_ratio_list_)) # uncomment to see the minimum of the list with the number of words per song
# print(max(love_ratio_list_)) # uncomment to see the maximum of the list with the number of words per song

# love_ratio_score()

# Creating the regularization step for the length attribute

    
#     for song in artist_s_cleaned_songs_list(artist):
#         cleaned_song_name = song.split("cleaned_")[1].split("~")[1]
#         id_ = song.split("cleaned_")[1].split("~")[0]
#         f = open(dir_given + '/Cleaned_Songs/' + song , 'rb')
#         for sentence in f.readlines():
#             this_sentence = sentence.decode('utf-8')            
#             raw_text += this_sentence
#         words = raw_text.split(" ")
#         for item in words:
#             if item in love_words_list_:
#                 counter_for_love_words += 1
#                 len_counter_for_love_words = len(love_words_list_)
#                 love_ratio = counter_for_love_words/float(len_counter_for_love_words)
#                 love_ratio = round(love_ratio,4)
#                 love_ratio_list_.append(love_ratio)
            
#         regularization_step = (love_ratio - min_love_ratio_list_)/(max_love_ratio_list_ - min_love_ratio_list_) 
#         love_ratio_score = 1*regularization_step + 0*(1-regularization_step)
#         love_ratio_score = round(love_ratio_score,2)
# #         print(love_ratio_score) # uncomment to see the complexity of each song